In [6]:
import pandas as pd
import torch as tr
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import time
import uuid
import tqdm
#import Model
from sklearn.metrics import precision_recall_fscore_support


def load_csv_data():
    # CSV dosyalarını yükleyin
    train_df = pd.read_csv('train_combined.csv')
    valid_df = pd.read_csv('eval_combined.csv')
    test_df = pd.read_csv('test_combined.csv')
    
    # Veriyi PyTorch Tensor formatına çevirin
    train_data, train_labels = tr.tensor(train_df.iloc[:, :-1].values), tr.tensor(train_df.iloc[:, -1].values).long()
    valid_data, valid_labels = tr.tensor(valid_df.iloc[:, :-1].values), tr.tensor(valid_df.iloc[:, -1].values).long()
    test_data, test_labels = tr.tensor(test_df.iloc[:, :-1].values), tr.tensor(test_df.iloc[:, -1].values).long()

    return (train_data, train_labels), (valid_data, valid_labels), (test_data, test_labels)


class Train:
    def __init__(self, args):
        # Dosyadan veriyi yükle
        self.train, self.valid, self.test = load_csv_data()
        
        self.args = args
        self.net = Model.FC_STGNN_HAR(
            args.patch_size, args.conv_out, args.lstmhidden_dim, args.lstmout_dim,
            args.conv_kernel, args.hidden_dim, args.time_denpen_len, args.num_sensor,
            args.num_windows, args.moving_window, args.stride, args.decay, args.pool_choice,
            args.n_class
        )

        self.net = self.net.cuda() if tr.cuda.is_available() else self.net
        self.loss_function = nn.CrossEntropyLoss()
        self.optim = optim.Adam(self.net.parameters())

    def Train_batch(self):
        self.net.train()
        loss_ = 0
        data, label = self.train
        data = data.cuda() if tr.cuda.is_available() else data
        label = label.cuda() if tr.cuda.is_available() else label

        self.optim.zero_grad()
        prediction = self.net(data)
        loss = self.loss_function(prediction, label)
        loss.backward()
        self.optim.step()
        loss_ = loss.item()
        return loss_

    def Train_model(self):
        epoch = self.args.epoch
        cross_accu = 0
        test_accu_ = []
        maf1_scores = []
        
        for i in range(epoch):
            loss = self.Train_batch()
            if i % self.args.show_interval == 0:
                accu_val = self.Cross_validation()

                if accu_val > cross_accu:
                    cross_accu = accu_val
                    test_accu, _, _, maf1_score = self.Prediction()
                    
                    maf1_scores.append(maf1_score)
                    test_accu_.append(test_accu)
                    self.save()
        
        np.save(f'experiment/HAR_{uuid.uuid4()}.npy', np.array([test_accu_, maf1_scores], dtype=object))

    def Cross_validation(self):
        self.net.eval()
        data, label = self.valid
        data = data.cuda() if tr.cuda.is_available() else data

        prediction = self.net(data)
        prediction = tr.argmax(prediction.detach().cpu(), -1)
        accu = self.accu_(prediction, label)
        return accu

    def Prediction(self):
        self.net.eval()
        data, label = self.test
        data = data.cuda() if tr.cuda.is_available() else data

        prediction = self.net(data)
        prediction = tr.argmax(prediction.detach().cpu(), -1)
        accu = self.accu_(prediction, label)
        maf1 = self.maf1(predicted=prediction, real=label)
        return accu, prediction, label, maf1

    def accu_(self, predicted, real):
        correct = (predicted == real).sum().item()
        return 100 * correct / real.size(0)

    def maf1(self, predicted, real):
        _, _, f1, _ = precision_recall_fscore_support(real, predicted, average="macro")
        return 100 * f1
    
    def save(self):
        tr.save(self.net.state_dict(), "HAR_PAMAP_trained")

    def load(self):
        self.net.load_state_dict(tr.load("HAR_PAMAP_trained"))

    def pertubate(self):
        self.net.eval()
        accus, mafs = list(), list()
        norm = 0
        num_features = 20
        for epsilon in tqdm.tqdm(np.linspace(0, 0.1, 15)):
            prediction_ = []
            real_ = []
            data, label = self.test
            data = data.cuda() if tr.cuda.is_available() else data
            label = label.cuda() if tr.cuda.is_available() else label
            data.requires_grad = True

            prediction = self.net(data)
            loss = self.loss_function(prediction, label)
            loss.backward()
            
            pertubation = epsilon * 2 * data.grad.data.sign()
            x_adv = tr.clamp(data + pertubation, -1, 1)
            prediction = self.net(x_adv)
            
            prediction = tr.argmax(prediction.detach().cpu(), -1)
            accu = self.accu_(prediction, label)
            maf1 = self.maf1(predicted=prediction, real=label)
            accus.append(accu)
            mafs.append(maf1)

        print(accus, mafs)


if __name__ == '__main__':
    from args import args
    args = args()

    def args_config_HAR(args):
        args.epoch = 50
        args.k = 1
        args.window_sample = 128
        args.decay = 0.7
        args.pool_choice = 'mean'
        args.moving_window = [2, 2]
        args.stride = [1, 2]
        args.lr = 1e-3
        args.batch_size = 100
        args.conv_kernel = 6
        args.patch_size = 64
        args.time_denpen_len = int(args.window_sample / args.patch_size)
        args.conv_out = 10
        args.num_windows = 2
        args.conv_time_CNN = 6
        args.lstmout_dim = 18
        args.hidden_dim = 16
        args.lstmhidden_dim = 48
        args.num_sensor = 9
        args.n_class = 6
        return args

    args = args_config_HAR(args)
    train = Train(args)
    # train.Train_model()
    train.load()
    train.pertubate()

ModuleNotFoundError: No module named 'args'